In [1]:
import os
import re
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/Taxml_Integers_Verified_12-11.csv', index_col=None, header=0,dtype={'item_name': str, 'description': str,'establishment_type': str, 'CAT_Name':str,'Integer':str},usecols=['item_name','description','establishment_type','CAT_Name','Integer'])
#df = pd.read_excel('TrainingDataSet_Integer.xlsx', index_col=None, header=0,usecols=['restaurant_UUID','item_uuid','item_name','description','establishment_type','merchant_type_analytics','CAT_Name'])
df_phase1 = pd.read_excel('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/Phase1SampleSet.xlsx', index_col=None, header=0,dtype={'item_name': str, 'description': str,'establishment_type': str, 'CAT_Name':str,'Integer':str},usecols=['item_name','description','establishment_type','CAT_Name','Integer'])
#read input from cicd data into dataframe
data_tagged=pd.read_csv('../data/retrain_data/data_tagged.csv', usecols = ['item_name','description','establishment_type','CAT_Name','Integer'])
#removing all duplicate rows
data_tagged=data_tagged.drop_duplicates(subset=['item_name','establishment_type','CAT_Name','Integer'],keep='first')
#fresh data of cicd
print(data_tagged.shape)
data_retrain=pd.read_csv('../data/retrain_data/data_latest_retraing_jul_aug.csv',usecols = ['item_name','description','establishment_type','CAT_Name','Integer'])
print(data_retrain.shape)
df_cat = pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/MASTER_Tax_Category_Mapping.csv',dtype={'CAT_Name':str,'target_new': str},usecols=['CAT_Name','target_new'])


(34171, 5)
(101602, 5)


In [2]:
df2=df[df['Integer']!='1']
df_phase1=df_phase1[df_phase1['Integer']!='1']

In [3]:
df2.shape,df_phase1.shape

((169945, 5), (135569, 5))

In [4]:
df2['target']=df2['CAT_Name']+":"+df2['Integer']
df_phase1['target']=df_phase1['CAT_Name']+":"+df_phase1['Integer']
data_tagged['target']= data_tagged['CAT_Name'] + ":" + data_tagged['Integer']
data_retrain['target']= data_retrain['CAT_Name'] + ":" + data_retrain['Integer']

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [82]:
final_df=pd.concat([df2, df_phase1, data_tagged, data_retrain])

In [83]:
final_df = final_df.reset_index(drop=True)

In [109]:
mergedStuff = pd.merge(final_df, df_cat, on=['CAT_Name'], how='left')
mergedStuff.head()

,item_name,description,establishment_type,CAT_Name,Integer,target,target_new
0,Captain Morgan White Rum.1.75L Bottle,NaN,GROCERY,CAT_LIQUOR,535,CAT_LIQUOR:535,CAT_LIQUOR:535
1,D’USSÉ® VSOP Cognac.375ml Bottle,NaN,GROCERY,CAT_LIQUOR,535,CAT_LIQUOR:535,CAT_LIQUOR:535
2,Jim Beam Honey Bourbon Whiskey.750ml Bottle,NaN,GROCERY,CAT_LIQUOR,535,CAT_LIQUOR:535,CAT_LIQUOR:535
3,Ardbeg Scotch Uigeadail. 750ml Bottle,NaN,GROCERY,CAT_LIQUOR,535,CAT_LIQUOR:535,CAT_LIQUOR:535
4,Jameson Irish Whiskey.1.75L Bottle Size,NaN,GROCERY,CAT_LIQUOR,535,CAT_LIQUOR:535,CAT_LIQUOR:535


In [119]:
final_merge=mergedStuff[mergedStuff['target_new'].notnull()]

In [260]:
#replace_merge=mergedStuff[mergedStuff['target_new'].isnull()]
#replace_merge.to_csv('replace_merge.csv')
#replace_merge=pd.read_csv('replace_merge.csv',index_col=None,header=0)
#replace_merge=replace_merge.drop(['Unnamed: 0'], axis=1)
#replace_merge=replace_merge.dropna()
def replace_column(df):
    s=''
    for index,rows in df_cat.iterrows():
        if rows['CAT_Name'] in str(df):
            print( rows['CAT_Name'],str(df))
            s=rows['target_new']
            return s
#replace_merge['target_new']=replace_merge['CAT_Name'].apply(replace_column)


In [284]:
historical=pd.concat([final_merge,replace_merge])

In [285]:
historical.shape

(432102, 7)

In [322]:
final_historical=pd.concat([historical,historical_cicd])

In [328]:
final_historical.shape

(524601, 7)

In [323]:
category_count = final_historical.groupby(['target_new'],sort=False).agg({'target_new':'count'})
category_count.rename(columns={'target_new':'count'},inplace=True)
category_count.sort_values('count',ascending=False)

,count
target_new,
CAT_TPP:531,70880
CAT_LIQUOR:535,51753
"CAT_CANDY,TEMP_COLD:108,3",39254
"CAT_PREPARED_FOOD,TEMP_HEATED:101,1",35404
"CAT_PREPACKAGED_FOOD,TEMP_COLD:106,3",28123
...,...
CAT_PREPACKAGED_FOOD_TOMATO_PUREE:745,3
CAT_PRESCRIPTION_DRUGS:520,2
CAT_POSTAGE:527,2


In [331]:
final_df=final_historical.drop_duplicates(subset=['item_name','description','establishment_type','target_new'],ignore_index=True)
print(final_df.shape)
final_df_without_duplicate=final_df.drop_duplicates(subset=['item_name','description','establishment_type'],keep=False,ignore_index=True)
print(final_df_without_duplicate.shape)
final_df_with_duplicate=final_df[final_df.duplicated(subset=['item_name','description','establishment_type'],keep=False)]
print(final_df_with_duplicate.shape)
final_df_without_duplicate.rename(columns = {'item_name':'Item','description':'Description'
                              }, inplace = True)
final_df_without_duplicate.to_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/historical_data_24_11_22.csv')

(383280, 7)
(381051, 7)
(2229, 7)


/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [330]:
final_df_without_duplicate

,item_name,description,establishment_type,CAT_Name,Integer,target,target_new
0,Captain Morgan White Rum.1.75L Bottle,NaN,GROCERY,CAT_LIQUOR,535,CAT_LIQUOR:535,CAT_LIQUOR:535
1,D’USSÉ® VSOP Cognac.375ml Bottle,NaN,GROCERY,CAT_LIQUOR,535,CAT_LIQUOR:535,CAT_LIQUOR:535
2,Jim Beam Honey Bourbon Whiskey.750ml Bottle,NaN,GROCERY,CAT_LIQUOR,535,CAT_LIQUOR:535,CAT_LIQUOR:535
3,Ardbeg Scotch Uigeadail. 750ml Bottle,NaN,GROCERY,CAT_LIQUOR,535,CAT_LIQUOR:535,CAT_LIQUOR:535
4,Jameson Irish Whiskey.1.75L Bottle Size,NaN,GROCERY,CAT_LIQUOR,535,CAT_LIQUOR:535,CAT_LIQUOR:535
...,...,...,...,...,...,...,...
381046,Joy · Color Cups (18 pieces),18 pieces,GROCERY,"CAT_CANDY,TRAIT_FLOUR,TEMP_HEATED","108,61,1","CAT_CANDY,TRAIT_FLOUR,TEMP_COLD:108,61,3","CAT_CANDY,TRAIT_FLOUR,TEMP_COLD:108,61,3"
381047,Joy · Ice Cream Cups (48 cups),48 cups,GROCERY,"CAT_CANDY,TRAIT_FLOUR,TEMP_HEATED","108,61,1","CAT_CANDY,TRAIT_FLOUR,TEMP_COLD:108,61,3","CAT_CANDY,TRAIT_FLOUR,TEMP_COLD:108,61,3"
381048,Fresh Michellada Mix (16 oz),Fresh michellada mix in 4 choices of flavor. U...,CONVENIENCE,"CAT_PREPARED_DRINK,TEMP_HEATED","114,1","CAT_PREPARED_DRINK,TEMP_UNHEATED:114,2","CAT_PREPARED_DRINK,TEMP_UNHEATED:114,2"
381049,Clamato,Clam and tomato juice beverage used to make cl...,CONVENIENCE,"CAT_JUICE,TRAIT_PCT_100,TEMP_HEATED","110,51,1","CAT_JUICE,TRAIT_PCT_100,TEMP_COLD:110,51,3","CAT_JUICE,TRAIT_PCT_100,TEMP_COLD:110,51,3"


In [326]:
print(final_df_without_duplicate.shape,final_df_with_duplicate.shape)
sorted_df5= final_df_with_duplicate.sort_values(by=['item_name','description','establishment_type'], ascending=True)#need to merge

(381051, 7) (2229, 7)


In [327]:
sorted_df5.to_csv('phase1_duplicate.csv')

In [304]:
#read input from cicd data into dataframe
data_cicd_backup=pd.read_csv('../data/retrain_data/Backup of TaxML-CICD_12_Aug_2022 - Prod_Data.csv', usecols = ['Item','Description','establishment_type','Confidence Score','Agent Corrected CAT Name', 'Agent Corrected Integer','CAT NAME_ ValidationScore [0-100]','Integer_ValidationScore[0-100]'])
#low conf and correcr predicted data
data_cicd_low_conf_backup=data_cicd_backup[(data_cicd_backup['Confidence Score']<0.6)]
data_cicd_low_conf_backup=data_cicd_low_conf_backup[(data_cicd_low_conf_backup['CAT NAME_ ValidationScore [0-100]']==100) &(data_cicd_low_conf_backup['Integer_ValidationScore[0-100]']==100)]
#misclassified data                                        
data_cicd_misclassification_backup=data_cicd_backup[(data_cicd_backup['CAT NAME_ ValidationScore [0-100]']!=100) &(data_cicd_backup['Integer_ValidationScore[0-100]']!=100)]
data_cicd_latest_backup=data_cicd_low_conf_backup.append(data_cicd_misclassification_backup)
data_cicd_latest_backup=data_cicd_latest_backup[['Item','Description','establishment_type','Agent Corrected CAT Name', 'Agent Corrected Integer']]
#remove empty rows from dataframe
data_cicd_latest_backup.dropna(how='all',inplace=True)
#remove rows having empty 'Agent Corrected CAT Name', ''Agent Corrected Integer'
data_cicd_latest_backup.dropna(subset=['Agent Corrected CAT Name', 'Agent Corrected Integer'],inplace=True)
#removing all duplicate rows
data_cicd_latest_backup=data_cicd_latest_backup.drop_duplicates(subset=['Item','establishment_type','Agent Corrected CAT Name', 'Agent Corrected Integer'],keep='first')
#print(data_cicd_latest_backup.shape)
#fresh data of cicd
data_cicd_without_duplicate_backup=data_cicd_latest_backup.drop_duplicates(subset=['Item','establishment_type'],keep=False)
print(data_cicd_without_duplicate_backup.shape)

(92379, 5)


In [305]:
data_cicd_without_duplicate_backup.rename(columns = {'Item':'item_name','Description':'description','Agent Corrected CAT Name':'CAT_Name', 'Agent Corrected Integer':'Integer',
                              }, inplace = True)
data_cicd_without_duplicate_backup['target']= data_cicd_without_duplicate_backup['CAT_Name'] + ":" + data_cicd_without_duplicate_backup['Integer']

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [306]:
data_cicd_without_duplicate_backup

,item_name,description,establishment_type,CAT_Name,Integer,target
14,Kalamata Jumbo Olives (1 lb),NaN,GROCERY,CAT_PREPACKAGED_FOOD_PICKLES,736,CAT_PREPACKAGED_FOOD_PICKLES:736
15,Organic Thyme (1 bunch),NaN,GROCERY,CAT_FRUIT_VEG_PLANTS,552,CAT_FRUIT_VEG_PLANTS:552
16,Aquaphor Advanced Protection (1.75 oz),NaN,GROCERY,CAT_MEDICATED_ITEMS,525,CAT_MEDICATED_ITEMS:525
17,Unisom Sleepgels (8 ct),NaN,GROCERY,CAT_OTC_MEDICATION,521,CAT_OTC_MEDICATION:521
18,Advil Coated Caplets (50 ct),NaN,GROCERY,CAT_OTC_MEDICATION_IBUPROFEN,768,CAT_OTC_MEDICATION_IBUPROFEN:768
...,...,...,...,...,...,...
611445,Rumple Minze Peppermint Schnapps (750 mL),If you like it strong you've come to the right...,LIQUOR,"CAT_SOFT_DRINK,CONTAINER_BOTTLED,TEMP_HEATED","112,11,1","CAT_SOFT_DRINK,CONTAINER_BOTTLED,TEMP_HEATED:1..."
613154,Darigold Butter 1 Pound,NaN,CONVENIENCE,CAT_PREPACKAGED_FOOD_BUTTER,717,CAT_PREPACKAGED_FOOD_BUTTER:717
613155,barkTHINS Dark Chocolate Pretzel with Sea Salt...,NaN,CONVENIENCE,CAT_PREPACKAGED_FOOD_SNACK_COOKIES,749,CAT_PREPACKAGED_FOOD_SNACK_COOKIES:749
613156,"Rickard's Red, 473 Ml (5.2% ABV)",NaN,CONVENIENCE,CAT_WINE,534,CAT_WINE:534


In [307]:
mergedStuff_cicd= pd.merge(data_cicd_without_duplicate_backup, df_cat, on=['CAT_Name'], how='left')
mergedStuff_cicd.head()

,item_name,description,establishment_type,CAT_Name,Integer,target,target_new
0,Kalamata Jumbo Olives (1 lb),NaN,GROCERY,CAT_PREPACKAGED_FOOD_PICKLES,736,CAT_PREPACKAGED_FOOD_PICKLES:736,CAT_PREPACKAGED_FOOD_PICKLES:736
1,Organic Thyme (1 bunch),NaN,GROCERY,CAT_FRUIT_VEG_PLANTS,552,CAT_FRUIT_VEG_PLANTS:552,CAT_FRUIT_VEG_PLANTS:552
2,Aquaphor Advanced Protection (1.75 oz),NaN,GROCERY,CAT_MEDICATED_ITEMS,525,CAT_MEDICATED_ITEMS:525,CAT_MEDICATED_ITEMS:525
3,Unisom Sleepgels (8 ct),NaN,GROCERY,CAT_OTC_MEDICATION,521,CAT_OTC_MEDICATION:521,CAT_OTC_MEDICATION:521
4,Advil Coated Caplets (50 ct),NaN,GROCERY,CAT_OTC_MEDICATION_IBUPROFEN,768,CAT_OTC_MEDICATION_IBUPROFEN:768,CAT_OTC_MEDICATION_IBUPROFEN:768


In [308]:
final_merge_cicd=mergedStuff_cicd[mergedStuff_cicd['target_new'].notnull()]

In [315]:
final_merge_cicd.shape,replace_merge_cicd.shape

((88174, 7), (9811, 7))

In [313]:
replace_merge_cicd=mergedStuff_cicd[mergedStuff_cicd['target_new'].isnull()]
replace_merge_cicd['target_new']=replace_merge_cicd['CAT_Name'].apply(replace_column)
#replace_merge_cicd.to_csv('replace_merge_cicd.csv')
#replace_merge_cicd=pd.read_csv('replace_merge_cicd.csv',index_col=None,header=0)
#replace_merge_cicd=replace_merge_cicd.drop(['Unnamed: 0'], axis=1)
#replace_merge_cicd=replace_merge_cicd.dropna()

CAT_WINE CAT_WINE,TEMP_HEATED
CAT_LIQUOR CAT_LIQUOR,TEMP_COLD
CAT_BEER CAT_BEER,TEMP_COLD
CAT_BEER CAT_BEER,TEMP_COLD
CAT_BEER CAT_BEER,TEMP_COLD
CAT_BEER CAT_BEER,TEMP_COLD
CAT_BEER CAT_BEER,TEMP_COLD
CAT_BEER CAT_BEER,TEMP_COLD
CAT_BEER CAT_BEER,TEMP_COLD
CAT_PREPACKAGED_FOOD_SNACK_CHIPS CAT_PREPACKAGED_FOOD_SNACK_CHIPS,TEMP_UNHEATED
CAT_LIQUOR CAT_LIQUOR,TEMP_UNHEATED
CAT_CHOCOLATE CAT_CHOCOLATE,TEMP_UNHEATED
CAT_CHOCOLATE CAT_CHOCOLATE,TEMP_UNHEATED
CAT_BEER CAT_BEER,TEMP_COLD


/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [320]:
historical_cicd=pd.concat([final_merge_cicd,replace_merge_cicd])

In [321]:
historical_cicd

,item_name,description,establishment_type,CAT_Name,Integer,target,target_new
0,Kalamata Jumbo Olives (1 lb),NaN,GROCERY,CAT_PREPACKAGED_FOOD_PICKLES,736,CAT_PREPACKAGED_FOOD_PICKLES:736,CAT_PREPACKAGED_FOOD_PICKLES:736
1,Organic Thyme (1 bunch),NaN,GROCERY,CAT_FRUIT_VEG_PLANTS,552,CAT_FRUIT_VEG_PLANTS:552,CAT_FRUIT_VEG_PLANTS:552
2,Aquaphor Advanced Protection (1.75 oz),NaN,GROCERY,CAT_MEDICATED_ITEMS,525,CAT_MEDICATED_ITEMS:525,CAT_MEDICATED_ITEMS:525
3,Unisom Sleepgels (8 ct),NaN,GROCERY,CAT_OTC_MEDICATION,521,CAT_OTC_MEDICATION:521,CAT_OTC_MEDICATION:521
4,Advil Coated Caplets (50 ct),NaN,GROCERY,CAT_OTC_MEDICATION_IBUPROFEN,768,CAT_OTC_MEDICATION_IBUPROFEN:768,CAT_OTC_MEDICATION_IBUPROFEN:768
...,...,...,...,...,...,...,...
9747,Joy · Color Cups (18 pieces),18 pieces,GROCERY,"CAT_CANDY,TRAIT_FLOUR,TEMP_HEATED","108,61,1","CAT_CANDY,TRAIT_FLOUR,TEMP_COLD:108,61,3","CAT_CANDY,TRAIT_FLOUR,TEMP_COLD:108,61,3"
9748,Joy · Ice Cream Cups (48 cups),48 cups,GROCERY,"CAT_CANDY,TRAIT_FLOUR,TEMP_HEATED","108,61,1","CAT_CANDY,TRAIT_FLOUR,TEMP_COLD:108,61,3","CAT_CANDY,TRAIT_FLOUR,TEMP_COLD:108,61,3"
9789,Fresh Michellada Mix (16 oz),Fresh michellada mix in 4 choices of flavor. U...,CONVENIENCE,"CAT_PREPARED_DRINK,TEMP_HEATED","114,1","CAT_PREPARED_DRINK,TEMP_UNHEATED:114,2","CAT_PREPARED_DRINK,TEMP_UNHEATED:114,2"
9790,Clamato,Clam and tomato juice beverage used to make cl...,CONVENIENCE,"CAT_JUICE,TRAIT_PCT_100,TEMP_HEATED","110,51,1","CAT_JUICE,TRAIT_PCT_100,TEMP_COLD:110,51,3","CAT_JUICE,TRAIT_PCT_100,TEMP_COLD:110,51,3"
